In [1]:
import numpy as np
import pandas as pd
import rasterio
import os
import matplotlib.pyplot as plt
from pyproj import Transformer

In [2]:
path_to_bps_data = "/home/adam/cloud/gdrive/postdoc/landfire_BPS_2020_102323"
path_to_bps_tiff = os.path.join(path_to_bps_data,"BPS_codes_study_area_all_values.tif")
bps_codes = pd.read_csv('bps_codes.csv')

## Functions

In [4]:
def create_raster_mask(input_raster_path,values_list,new_value,output_raster_path):

    # Open the raster
    with rasterio.open(input_raster_path) as src:
        # Read the data
        data = src.read(1)
        data = data.astype("uint16")

        # Create a mask where values are in the list
        mask = np.isin(data, values_list)

        # Set cells in the mask to 1 and others to nodata
        nodata = 9999
        data[mask] = new_value
        data[~mask] = nodata

        # Save the modified data to a new raster file
        with rasterio.open(output_raster_path, 'w', driver='GTiff', height=src.height, width=src.width,
                           count=1, dtype=data.dtype, crs=src.crs, transform=src.transform, nodata=nodata) as dst:
            dst.write(data, 1)

        print(f"Set values in {values_list} to {new_value} and saved to {output_raster_path}")
        
def extract_value_at_lat_lon(raster_path, lat, lon, null_value):
    """Extract the closest non-null raster value at specific latitude and longitude after transforming to EPSG:5070."""

    # Transform lat, lon from EPSG:4326 (WGS 84) to EPSG:5070 (NAD83 / Conus Albers)
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:5070")
    lon_transformed, lat_transformed = transformer.transform(lat, lon)

    with rasterio.open(raster_path) as src:
        # Transform the transformed coordinates to row, col
        row, col = src.index(lon_transformed, lat_transformed)
        
        # Start spiral search for the closest non-null value
        directions = [(0,1), (1,0), (0,-1), (-1,0)]  # Right, Down, Left, Up
        max_distance = max(src.height, src.width)  # Max search radius
        for distance in range(1, max_distance):
            for i in range(4):  # For each direction
                for j in range(distance):  # Move 'distance' steps in that direction
                    row_offset, col_offset = directions[i]
                    new_row, new_col = row + row_offset * j, col + col_offset * j
                    
                    # Check if the new coordinates are within the raster bounds
                    if 0 <= new_row < src.height and 0 <= new_col < src.width:
                        value = src.read(1)[new_row, new_col]
                        if value != null_value:  # Assuming np.nan represents the null value
                            return value
                # Rotate direction
                directions = directions[1:] + directions[:1]
        raise ValueError("Could not find a non-null value in the vicinity of the specified coordinates.")

## Get raster values for dry and moist MCF types

In [5]:
bps_codes.head()

# Define the BPS codes that Mallek et al., 2013 defined as part of the moist mixed conifer zone
mmc_bps_codes = [10210,10280,10220]

# Define the BPS codes that Mallek et al., 2013 defined as part of the dry mixed conifer zone
dmc_bps_codes = [10270]

# Print raster values for moist mixed conifer forest
mmc_values = list(bps_codes.loc[bps_codes['BPS_CODE'].isin(mmc_bps_codes)]['VALUE'])
dmc_values = list(bps_codes.loc[bps_codes['BPS_CODE'].isin(dmc_bps_codes)]['VALUE'])
print("moist MCF values:",mmc_values)
print("dry MCF values:",dmc_values)

moist MCF values: [549, 550, 552, 583, 585, 762, 763, 765, 795, 796, 798, 932, 1052]
dry MCF values: [551, 584, 764, 797, 911, 931]


## Write the new raster for dry and wet MCF

In [92]:
create_raster_mask(input_raster_path = path_to_bps_tiff,values_list = mmc_values,new_value = 2, output_raster_path=os.path.join(path_to_bps_data,"mmc_raster.tif"))
create_raster_mask(input_raster_path = path_to_bps_tiff,values_list = dmc_values,new_value = 1, output_raster_path=os.path.join(path_to_bps_data,"dmc_raster.tif"))

Set values in [549, 550, 552, 583, 585, 762, 763, 765, 795, 796, 798, 932, 1052] to 2 and saved to /home/adam/cloud/gdrive/postdoc/landfire_BPS_2020_102323/mmc_raster.tif
Set values in [551, 584, 764, 797, 911, 931] to 1 and saved to /home/adam/cloud/gdrive/postdoc/landfire_BPS_2020_102323/dmc_raster.tif


In [5]:
with rasterio.open('/home/adam/cloud/gdrive/postdoc/landfire_BPS_2020_102323/dmc_raster.tif') as src:
    no_data_value = src.nodata
    
with rasterio.open('/home/adam/cloud/gdrive/postdoc/landfire_BPS_2020_102323/dmc_raster.tif') as src:
    data_type = src.dtypes[0]  # assuming a single band; for multi-band rasters, this will give the data type of the first band

print("Data type:",data_type)

print("No data value:",no_data_value)

Data type: uint16
No data value: 9999.0


In [6]:
# with rasterio.open('/home/adam/cloud/gdrive/postdoc/landfire_BPS_2020_102323/mmc_raster.tif') as src:
#     # Read the raster data as a numpy array
#     raster_data = src.read(1)  # Change the band number as needed

# histogram, bins = np.histogram(raster_data, bins=5, range=(0, 3))  # Adjust the bins and range as needed

# plt.hist(raster_data.ravel(), bins=5, range=(0, 5), color='blue', alpha=0.7)
# plt.xlabel('Pixel Value')
# plt.ylabel('Frequency')
# plt.title('Histogram of Raster Values')
# plt.show()

## Merge the dry and mixed rasters

In [15]:
import rasterio
from rasterio.merge import merge

# Define the filepaths for your input rasters and output raster
raster1_path = os.path.join(path_to_bps_data,"mmc_raster.tif")
raster2_path = os.path.join(path_to_bps_data,"dmc_raster.tif")
merged_raster_path = os.path.join(path_to_bps_data,'mmc_and_dmc_bps.tif')

# Open the rasters
with rasterio.open(raster1_path) as src1, rasterio.open(raster2_path) as src2:
    # Merge the two rasters
    merged_data, merged_transform = merge([src1, src2])
    
    
    # Write the merged raster to a new file
    with rasterio.open(merged_raster_path, 'w', 
                       driver='GTiff',
                       height=merged_data.shape[1],
                       width=merged_data.shape[2],
                       count=merged_data.shape[0],
                       dtype=merged_data.dtype,
                       crs=src1.crs,
                       nodata=9999,
                       transform=merged_transform) as dst:
        dst.write(merged_data)

print(f"Merged raster saved to: {merged_raster_path}")

Merged raster saved to: /home/adam/cloud/gdrive/postdoc/landfire_BPS_2020_102323/mmc_and_dmc_bps.tif


### See coordinate reference system and no data value for the merged raster

In [16]:
forest_type_path = '/home/adam/cloud/gdrive/postdoc/landfire_BPS_2020_102323/mmc_and_dmc_bps.tif'
with rasterio.open(forest_type_path) as src:
    crs = src.crs
print("CRS:",crs)

with rasterio.open(forest_type_path) as src:
    no_data_value = src.nodata
print(no_data_value)

CRS: EPSG:5070


## Check if point is MMC or DMC

In [24]:
raster_path = forest_type_path
lat = 35.8846892836225  # Replace with your latitude
lon = -118.595324757914 # Replace with your longitude



value = extract_value_at_lat_lon(raster_path, lat, lon, null_value=9999)
print(value)

2


## Make polygon from the forest types

In [ ]:
# import rasterio
# import numpy as np
# from scipy.ndimage import label
# from shapely.geometry import mapping, shape
# from shapely.ops import cascaded_union
# from skimage.measure import find_contours

# def raster_to_polygon_contour(input_raster, mask_value, alpha=0.05):
#     """
#     Create a polygon around the unmasked values in a raster.
#     :param input_raster: Path to the input raster file.
#     :param mask_value: Value used in the raster to represent masked data.
#     :param alpha: Alpha level for the probability contour. 0.05 for 95% contour.
#     :return: A Shapely polygon.
#     """
    
#     # Load the raster data
#     with rasterio.open(input_raster) as src:
#         data = src.read(1)
#         transform = src.transform
    
#     # Create a binary array (1 for unmasked, 0 for masked)
#     binary_array = (data != mask_value).astype(int)
    
#     # Find connected components
#     labeled_array, num_features = label(binary_array)
    
#     polygons = []
    
#     # For each connected component, generate a polygon
#     for feature_id in range(1, num_features + 1):
        
#         component_array = (labeled_array == feature_id).astype(float)
        
#         # Find contours at half intensity
#         contours = find_contours(component_array, 0.5)
        
#         # Convert each contour to a polygon
#         for contour in contours:
#             # Transform from pixel coordinates to spatial coordinates
#             spatial_contour = [~transform * (y, x) for x, y in contour]
#             poly = shape({
#                 "type": "Polygon",
#                 "coordinates": [spatial_contour]
#             })
            
#             polygons.append(poly)
    
#     # Merge all polygons
#     merged_polygon = cascaded_union(polygons)
    
#     return merged_polygon

# input_raster_path = 'path_to_your_raster.tif'
# mask_value = 0  # Replace with your mask value

# # Generate the polygon
# polygon = raster_to_polygon_contour(input_raster_path, mask_value)

# print(polygon)